# Init

In [1]:
import yaml, pprint

try:
    reload(stdcell)
except NameError:
    import stdcell
from stdcell import *

# Read source YAML file

In [2]:
# Load nsxlib YAML file
with open("../nsxlib/nsxlib.yaml","r") as f:
    nsxlib = yaml.load(f, Loader=yaml.FullLoader)

# Support code

In [3]:
def elem_from_comp(comp, cell_width):
    comp_layer = comp["layer"]
    comp_net = comp["net"]
    comp_type = comp["type"]

    # Skip template components
    if ((comp_net in ("vss", "vdd"))
        and (comp_type == "Horizontal")
        and (comp["layer"] == "METAL1")
       ):
        y = comp["y"]
        if ((comp["width"] != 2400)
            or (comp["dx"]["source"] != 0)
            or (comp["dx"]["target"] != cell_width)
            or ((comp_net == "vss") and (y != 1200))
            or ((comp_net == "vdd") and (y != 18800))
           ):
            raise ValueError("Wrong DC rail dimension")
        return comp_net, None
    elif comp_layer == "NWELL":
        if ((comp_type != "Horizontal")
            or (comp["width"] != 12000)
            or (comp["y"] != 15600)
            or (comp["dx"]["source"] != -600)
            or (comp["dx"]["target"] != (cell_width + 600))
           ):
            raise ValueError("Wrong NWELL dimensions")
        return comp_net, None

    # Add type specific kwargs
    if comp_layer in Wire.layers:
        if comp_type == "Horizontal":
            start = comp["dx"]["source"]
            end = comp["dx"]["target"]
            x = (min(start, end), max(start, end))
            y = comp["y"]
        elif comp_type == "Vertical":
            start = comp["dy"]["source"]
            end = comp["dy"]["target"]
            x = comp["x"]
            y = (min(start, end), max(start, end))
        else:
            raise ValueError("Unknown component type {}".format(comp_type))
        width = comp["width"]

        elem = Wire(comp_layer, x, y, comp["width"], external=comp["external"])
    elif comp_layer in Via.layer2bottom:
        elem = Via(Via.layer2bottom[comp_layer], Via.layer2top[comp_layer], comp["x"], comp["y"], comp["width"])
    elif comp_layer in Device.layer2type:
        assert comp["type"] == "Vertical"

        dy_source = comp["dy"]["source"]
        dy_target = comp["dy"]["target"]
        elem = Device(
            type_=Device.layer2type[comp_layer],
            x=comp["x"], y=(dy_source + dy_target)//2, l=comp["width"], w=abs(dy_target - dy_source),
            direction="vertical",
        )
    else:
        raise ValueError("Unsupported layer {}".format(comp_layer))

    return comp_net, elem

In [4]:
class CellConverter:
    def __init__(self, name, celldata, print_name=False):
        if print_name:
            print(name)
        self.name = name

        boundarydata = celldata["boundary"]
        assert (boundarydata["x1"] == 0) and (boundarydata["y1"] == 0)
        netsdata = celldata["nets"]
        ports = set(filter(lambda net: netsdata[net]["external"], netsdata.keys()))
        self.cell = cell = StdCell(name, boundarydata["x2"], boundarydata["y2"])
        
        # Create Elem for each component
        unhandled = []
        netelems = []
        for comp in celldata["components"]:
            try:
                net, elem = elem_from_comp(comp, boundarydata["x2"])
            except ValueError as e:
                unhandled.append((e, comp))
            else:
                if elem:
                    cell.add_elem(elem, net)

        if len(unhandled) != 0:
            print("Unhandled components for {}:".format(name))
            for e, comp in unhandled:
                print("- {} ({})".format(comp, str(e)))

        # Get final net names
        self.merged = cell.finalize()["merged"]

        assert ports == cell.ports

    def to_dict(self):
        cell = self.cell
        return {
            "width": cell.width,
            "height": cell.height,
            "ports": cell.ports,
            "nets": cell.nets,
            "nmerged": self.merged,
        }

# Convert cells

In [5]:
convcells = [CellConverter(name, data, print_name=False) for name, data in nsxlib.items()]
c4mlib = {
    "trackpitch": {"horizontal": 2000, "vertical": 2000},
    "cellheight": 10,
    "cells": [convcell.cell for convcell in convcells],
}
merged = [(convcell.cell.name, convcell.merged) for convcell in convcells]
merged.sort()
pprint.pprint(merged)

[('a2_x2', 3),
 ('a2_x4', 3),
 ('a3_x2', 4),
 ('a3_x4', 4),
 ('a4_x2', 5),
 ('a4_x4', 6),
 ('an12_x1', 4),
 ('an12_x4', 4),
 ('ao22_x2', 4),
 ('ao22_x4', 4),
 ('ao2o22_x2', 5),
 ('ao2o22_x4', 5),
 ('buf_x2', 2),
 ('buf_x4', 2),
 ('buf_x8', 2),
 ('inv_x1', 2),
 ('inv_x2', 2),
 ('inv_x4', 2),
 ('inv_x8', 2),
 ('mx2_x2', 4),
 ('mx2_x4', 4),
 ('mx3_x2', 5),
 ('mx3_x4', 5),
 ('na2_x1', 3),
 ('na2_x4', 5),
 ('na3_x1', 4),
 ('na3_x4', 6),
 ('na4_x1', 5),
 ('na4_x4', 5),
 ('nao22_x1', 4),
 ('nao22_x4', 4),
 ('nao2o22_x1', 5),
 ('nao2o22_x4', 5),
 ('nmx2_x1', 5),
 ('nmx2_x4', 4),
 ('nmx3_x1', 3),
 ('no2_x1', 4),
 ('no2_x4', 3),
 ('no3_x1', 4),
 ('no3_x4', 5),
 ('no4_x1', 5),
 ('no4_x4', 8),
 ('noa22_x1', 4),
 ('noa22_x4', 4),
 ('noa2a22_x1', 5),
 ('noa2a22_x4', 6),
 ('noa2a2a23_x1', 7),
 ('noa2a2a23_x4', 8),
 ('noa2a2a2a24_x1', 9),
 ('noa2a2a2a24_x4', 9),
 ('noa2ao222_x1', 7),
 ('noa2ao222_x4', 7),
 ('noa3ao322_x1', 9),
 ('noa3ao322_x4', 9),
 ('nts_x1', 3),
 ('nts_x2', 3),
 ('nxr2_x1', 3),
 ('n

# Write python source code

In [6]:
with open("flexlib.py", "w") as fpy:
    fpy.write("from celllib import StdCell, Wire, Via, Device\n\n")
    fpy.write("flexlib = [\n")
    for cell in c4mlib["cells"]:
        fpy.write(cell.python_code(level=1) + ",\n")
    fpy.write("]\n")

# Playground

In [7]:
#cellname = "inv_x1"
#cellname = "sff1r_x4"
#cellname = "a2_x2"
cellname = "na2_x1"
#cellname = "sff1_x4"
#cellname = "nxr2_x1"
#cellname = "noa2a22_x1"
conv = CellConverter(cellname, nsxlib[cellname])
c4mcell = conv.cell
print("Merged: {}".format(conv.merged))
print("Ports:")
for port in c4mcell.ports:
    print("- {}".format(port))
for net, elems in c4mcell.nets.items():
    print("Net {}:".format(net))
    for elem in elems:
        print("{}".format(elem.str_indent(0, "+ ", level_str="| ", net=net)))

Merged: 3
Ports:
- i1
- nq
- vss
- vdd
- i0
Net vss:
+ METAL1((1600,2400)-(2400,4200))
| + NDIF<->METAL1((2000,4000))
| | + NDIF((1000,3200)-(2200,6800))
+ PTIE<->METAL1((5000,1400))
| + PTIE((4400,600)-(6600,2200))
Net i0:
+ EXT_METAL1((1700,6000)-(2300,14000))
| + POLY<->METAL1((2000,10000))
| | + POLY((2600,7400)-(3000,13200))
| | | + pmos((2800,15000),l=400,w=4000)
| | | + nmos((2800,5000),l=400,w=4000)
| | + POLY((2300,9400)-(2900,10600))
| | + POLY((1800,9800)-(3000,10200))
Net i1:
+ EXT_METAL1((5700,6000)-(6300,14000))
| + POLY<->METAL1((6000,8000))
| | + POLY((5000,7800)-(6200,8600))
| | + POLY((5000,7800)-(5400,12600))
| | | + pmos((5200,15000),l=400,w=4000)
| | + POLY((4400,7600)-(6200,8000))
| | | + POLY((4400,7200)-(4800,8000))
| | | | + nmos((4600,5000),l=400,w=4000)
Net vdd:
+ METAL1((1600,15800)-(2400,17600))
| + PDIF<->METAL1((2000,16000))
| | + PDIF((1000,13200)-(2200,16800))
+ METAL1((5600,15800)-(6400,17600))
| + PDIF<->METAL1((6000,16000))
| | + PDIF((5800,13200)-(7

In [8]:
cell = c4mlib["cells"][0]
print(cell.python_code())

StdCell(
    name='a2_x2', width=10000, height=20000,
    nets={
        '_net0': [
            Wire('NDIF', 3600, (1600, 4600), 1200, False),
        ],
        '_net1': [
            Wire('METAL1', 3800, (4200, 16000), 600, False),
            Via('PDIF', 'METAL1', 3400, 16000, 200),
            Wire('METAL1', (1600, 3600), 4000, 600, False),
            Via('POLY', 'METAL1', 4000, 8000, 200),
            Wire('POLY', (4000, 7200), 8000, 400, False),
            Wire('POLY', 7200, (5600, 10400), 400, False),
            Device('pmos', 7200, 15000, 400, 8000, 'vertical', source_net='vdd', drain_net='q'),
            Device('nmos', 7200, 3100, 400, 3800, 'vertical', source_net='vss', drain_net='q'),
            Via('NDIF', 'METAL1', 1600, 4000, 200),
            Wire('NDIF', 1600, (1600, 4600), 1200, False),
            Wire('PDIF', 3600, (13400, 16600), 1200, False),
        ],
        'i0': [
            Wire('METAL1', 2000, (6200, 13800), 600, True),
            Via('POLY', 'METAL1'